# Arcanum Tag-Driven World System
This notebook contains the file layout and key scripts for the Tag-Driven World System.


## File layout
```
ServerScriptService/
  Modules/
    TagManager.lua
    Config.lua
    Handlers/
      TreasureChestHandler.lua
      DamageZoneHandler.lua
  InitTags.server.lua
README.md
```


## TagManager.lua (core)
The TagManager registers handlers and auto-applies them to existing and future tagged instances.


In [ ]:
-- TagManager.lua (ModuleScript)
local CollectionService = game:GetService("CollectionService")
local TagManager = {}

local handlers = {}
local applied = setmetatable({}, {__mode = "k"}) -- weak keys (instances)

local function safeRun(fn, ...)
    local ok, err = pcall(fn, ...)
    if not ok then
        warn("TagManager error:", err)
    end
end

function TagManager.registerHandler(tagName, handler)
    if handlers[tagName] then
        warn("Overwriting handler for tag:", tagName)
    end
    handlers[tagName] = handler

    -- apply to current instances with this tag
    for _, inst in ipairs(CollectionService:GetTagged(tagName)) do
        applied[inst] = applied[inst] or {}
        if not applied[inst][tagName] then
            applied[inst][tagName] = true
            safeRun(handler.setup, inst)
        end
    end

    -- listen for future instances
    CollectionService:GetInstanceAddedSignal(tagName):Connect(function(inst)
        applied[inst] = applied[inst] or {}
        if not applied[inst][tagName] then
            applied[inst][tagName] = true
            safeRun(handler.setup, inst)
        end
    end)

    -- listen for removed instances to teardown
    CollectionService:GetInstanceRemovedSignal(tagName):Connect(function(inst)
        if applied[inst] and applied[inst][tagName] then
            applied[inst][tagName] = nil
            if handler.teardown then
                safeRun(handler.teardown, inst)
            end
        end
    end)
end

function TagManager.hasTag(instance, tagName)
    return CollectionService:HasTag(instance, tagName)
end

function TagManager.addTag(instance, tagName)
    CollectionService:AddTag(instance, tagName)
end

function TagManager.removeTag(instance, tagName)
    CollectionService:RemoveTag(instance, tagName)
end

return TagManager


## TreasureChestHandler.lua
Loot roll and proximity prompt handling.


In [ ]:
-- TreasureChestHandler.lua (ModuleScript)
local CollectionService = game:GetService("CollectionService")

local TreasureChestHandler = {}

local LootTable = {
    {item = "Gold", min = 10, max = 50, weight = 70},
    {item = "Potion", min = 1, max = 1, weight = 20},
    {item = "RareGem", min = 1, max = 1, weight = 10}
}

local function rollLoot()
    local total = 0
    for _, v in ipairs(LootTable) do total = total + v.weight end
    local r = math.random(1, total)
    local sum = 0
    for _, v in ipairs(LootTable) do
        sum = sum + v.weight
        if r <= sum then
            local amount = math.random(v.min, v.max)
            return v.item, amount
        end
    end
end

local function givePlayerLoot(player, itemName, amount)
    local ls = player:FindFirstChild("leaderstats")
    if ls and ls:FindFirstChild(itemName) then
        ls[itemName].Value = ls[itemName].Value + amount
    else
        if ls then
            local stat = Instance.new("IntValue")
            stat.Name = itemName
            stat.Value = amount
            stat.Parent = ls
        end
    end
end

function TreasureChestHandler.setup(chest)
    if not chest then return end
    if not chest:IsA("Model") then return end

    if chest:GetAttribute("ChestConnected") then return end
    chest:SetAttribute("ChestConnected", true)

    local prompt = chest:FindFirstChildOfClass("ProximityPrompt")
    if not prompt then
        local primary = chest.PrimaryPart or chest:FindFirstChildWhichIsA("BasePart")
        if primary then
            prompt = Instance.new("ProximityPrompt")
            prompt.ActionText = "Open"
            prompt.ObjectText = chest.Name
            prompt.HoldDuration = 0.6
            prompt.Parent = primary
        else
            warn("TreasureChestHandler: chest has no PrimaryPart or BasePart to attach a prompt to:", chest:GetFullName())
            return
        end
    end

    prompt.Triggered:Connect(function(player)
        local item, amt = rollLoot()
        givePlayerLoot(player, item, amt)
        -- TODO: visual feedback (particles/sound) via RemoteEvent
        chest:Destroy()
    end)
end

function TreasureChestHandler.teardown(chest)
    -- optional cleanup
end

return TreasureChestHandler


## DamageZoneHandler.lua
Applies damage while player touches the tagged part.


In [ ]:
-- DamageZoneHandler.lua (ModuleScript)
local Players = game:GetService("Players")
local DAMAGE_PER_SECOND = 10

local DamageZone = {}

local function damageCharacter(character, dps)
    local humanoid = character and character:FindFirstChildOfClass("Humanoid")
    if humanoid and humanoid.Health > 0 then
        humanoid:TakeDamage(dps)
    end
end

function DamageZone.setup(part)
    if not part or not part:IsA("BasePart") then return end
    if part:GetAttribute("DamageConnected") then return end
    part:SetAttribute("DamageConnected", true)

    local touching = {}

    part.Touched:Connect(function(hit)
        local player = Players:GetPlayerFromCharacter(hit.Parent)
        if player and not touching[player] then
            touching[player] = true
            spawn(function()
                while touching[player] and player.Character and player.Character.Parent do
                    damageCharacter(player.Character, DAMAGE_PER_SECOND)
                    wait(1)
                end
            end)
        end
    end)

    part.TouchEnded:Connect(function(hit)
        local player = Players:GetPlayerFromCharacter(hit.Parent)
        if player then touching[player] = nil end
    end)
end

function DamageZone.teardown(part)
    -- optional cleanup
end

return DamageZone


## InitTags.server.lua
Registers handlers on server startup.


In [ ]:
-- InitTags.server.lua (Server Script)
local TagManager = require(script.Parent.Modules.TagManager)
local TreasureChestHandler = require(script.Parent.Modules.Handlers.TreasureChestHandler)
local DamageZoneHandler = require(script.Parent.Modules.Handlers.DamageZoneHandler)

-- Register handlers for tags
TagManager.registerHandler("TreasureChest", TreasureChestHandler)
TagManager.registerHandler("Lava", DamageZoneHandler)
TagManager.registerHandler("PoisonZone", DamageZoneHandler)


## README
See README.md for usage instructions.
